##### 작성일 : 2021.11.05(금)
##### 작성자  : 손해원

In [ ]:
import pandas as pd
from selenium import webdriver
import time
from tqdm import tqdm 

In [ ]:
driver = webdriver.Chrome('/home/haewon/chromedriver') #### 드라이버 경로 설정 

##### 검색어 설정 

In [ ]:
keyword = '김천%20맛집'
url = f'https://section.blog.naver.com/Search/Post.naver?pageNo=1&rangeType=PERIOD&orderBy=sim&startDate=2020-11-01&endDate=2021-11-05&keyword={keyword}'
driver.get(url)

##### url 수집

In [ ]:
### 최근 1년동안의 url만 모으자 
search_number = int(driver.find_element_by_class_name('search_number').text.replace(',','').replace('건','')) # 총 게시물 수 
hrefs = driver.find_elements_by_class_name('desc_inner')
url_list = []
num = 1 

for num in tqdm(range(1, int(search_number/7))):  # 총 게시물 수/7(한 페이지당 게시물 수)
    for i in range(len(hrefs)):
        time.sleep(0.5)
        href = driver.find_elements_by_class_name('desc_inner')[i].get_attribute('href')
        url_list.append(href)
    num += 1
    driver.get(f'https://section.blog.naver.com/Search/Post.naver?pageNo={num}&rangeType=PERIOD&orderBy=sim&startDate=2020-11-01&endDate=2021-11-05&keyword={keyword}')
    
    ##### 에러로 끝나도 당황하지 마세요 !!!!

In [ ]:
len(url_list)

##### 본문 스크랩

In [ ]:
driver.find_element_by_class_name('')

In [ ]:
title_list = []
writer_list = []
date_list = []
text_list = []
tag_list = []

for i in tqdm(range(len(url_list))):
    driver.get(url_list[i])
    element = driver.find_element_by_id("mainFrame") #iframe 태그 엘리먼트 찾기
    driver.switch_to.frame(element) #프레임 이동
    
    title = driver.find_element_by_class_name('pcol1').text # 제목 
    title_list.append(title)
    writer = driver.find_element_by_class_name('nick').text # 글쓴이
    writer_list.append(writer)
    time.sleep(1)
    try:
        date = driver.find_element_by_class_name('se_publishDate.pcol2').text # 작성일 
        date_list.append(date)
    except:
        date_list.append("NULL")
    try:
        text = driver.find_element_by_class_name('se-main-container').text.replace('\n',' ') # 본문 
        text_list.append(text)
    except:
        text_list.append("NULL")
    
    try: 
        tags = driver.find_elements_by_class_name('item.pcol2.itemTagfont._setTop')
        tags_dummy = []
        for j in range(len(tags)):
            tag = driver.find_elements_by_class_name('item.pcol2.itemTagfont._setTop')[j].text.replace('#','')  # 해쉬태그
            tags_dummy.append(tag)
        tag_list.append(tags_dummy)
    except:
        tag_list.append("NULL")
        
    

In [ ]:
len(title_list)

In [ ]:
len(writer_list)

In [ ]:
len(tag_list)

In [ ]:
len(writer_list)

In [ ]:
len(text_list)

##### 저장

In [69]:
blog = pd.DataFrame(
    {
        "source":"NAVER_BLOG",
        "url":url_list,
        "title":title_list,
        "writer":writer_list,
        "date":date_list,
        "main_text":text_list,
        "tag":tag_list
    }
)

In [70]:
blog

,source,title,writer,date,main_text,tag
0,NAVER_BLOG,믿고먹는 김천 맛집 리스트,오꼬밍\n(mjmj9209),2021. 7. 15. 8:33,유린기를 맛있게 잘 하기로 알아주는 김천 맛집을 가기 위해 온 가족이 함께 움직였어...,"[김천맛집, 화타]"
1,NAVER_BLOG,김천맛집 순대국 좋아하면 꼭 가야함!,뷰티,2021. 10. 19. 10:04,김천맛집 순대국 좋아하면 꼭 가야함! 최근데 다녀온 김천 맛집 리뷰입니다! 요즘 처...,"[김천맛집, 순대국맛집]"
2,NAVER_BLOG,"김천 맛집, 김천 삼겹살 맛집, 돼지만",트윙클\n(twinkle_ja),2021. 7. 12. 19:21,"지난 주말, 초복이였죠? 친정부모님 모시고 닭 먹으러 갈까 하다가 다들 고기를 ...","[김천맛집, 김천삼겹살, 김천삼겹살맛집, 돼지만, 돼지만김천점, 김천생삼겹살, 김천]"
3,NAVER_BLOG,"[경북 김천] 착한 짬뽕 : 저렴한 가격에 맛있고, 배도 둔둔, 마음도 든든 / ...",미미,2021. 6. 24. 8:00,안녕하세요. 왜때문에 맛있는 건 매끼니마다 먹고싶을까요? 🤣 얼마전 미식가 친구에게...,"[착한짬뽕, 김천맛집, 김천, 경북김천, 경북김천맛집, 김천짬뽕, 김천짜장면, 김천..."
4,NAVER_BLOG,上 김천 막창 맛집 : 대구반야월막창,하짱,2021. 11. 3. 15:22,#대구반야월막창 안녕하세요 하짱입니당 :-) 오늘은 제가 태어나서 처음가보고 완전 ...,"[김천막창, 김천막창맛집, 김천맛집, 신음동맛집, 대구반야월막창, 김천반야월막창, ..."
5,NAVER_BLOG,푸짐하고 맛있는 김천 직지사 맛집,꽁쥬,2021. 8. 26. 23:51,NULL,NULL


In [56]:
blog.to_csv('/home/haewon/바탕화면/naver_blog_1109_김천맛집.csv',index=False)